In [1]:
class ChlGlobal:
    '''
    class Chlsurf_OCCCI_comp(runname,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                 mapproj='pc',savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
                 
    
    self.ChlAfesom_surf_interp contains 2D dataset of 1x1 interpolated nanophytoplankton Chl.a
    self.DiaChlfesom_surf_interp contains 2D dataset of 1x1 interpolated diatom Chl.a
    self.Chl_total_interp = Chl_total contains 2D dataset of 1x1 interpolated sum of Chl.a
    self.unitfesom contains str of FESOM Chl.a unit
    '''
    
    def __init__(self,resultpath,savepath,mesh,matfileChlsurf,first_year,last_year,
                 mapproj='rob',runname='fesom',
                 savefig=False,output=False,plotting=True,verbose=False,Taylor=True):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.matfileChlsurf=matfileChlsurf
        self.verbose = verbose
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
        
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        matChl = spio.loadmat(self.matfileChlsurf, squeeze_me=True)
        
        lat         = np.arange(-89.5,90.,1.)
        lon         = np.arange(-179.5,180.,1.)
        latdic, londic = np.meshgrid(lat, lon)
        
        #annualchl   = np.log10(matChl['x'])
        OCCCIchla = matChl['x']
        OCCCIchla_log10 = np.log10(OCCCIchla)
        
        OCCCIlabel = 'OC-CCI'
        OCCCIunit = 'Chl.a [mg Chl m$^{-3}$]'          
        print(' !!! Satellite data only for 2012-2015 period !!!')
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(self.meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2        
        
        # load FESOM Nanophyto Chl.a data -------------------------------------------------------------------------------------        
        
        #ncFESOMChl = self.resultpath + '/PhyChl.fesom.1948.nc'  # Miriam: not needed?
        #!ncdump -h $ncFESOMChl
        
        PhyChlfesom = pf.get_data(self.resultpath, "PhyChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        PhyChlfesom_surf = PhyChlfesom[:,0]
        
        labelfesomNano = 'FESOM-REcoM Nanophyto Chl.a {0}-{1}'.format(self.fyear,self.lyear)        
        #unitfesomNano = 'Chl.a [mmol m$^{-3}$]' 

        # load FESOM Diatom Chl.a data -------------------------------------------------------------------------------------
        
        #ncFESOMDiaChl = self.resultpath + '/DiaChl.fesom.1948.nc'   # Miriam: not needed?
        #!ncdump -h $ncFESOMDiaChl
        
        DiaChlfesom = pf.get_data(self.resultpath, "DiaChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        labelfesomDia = 'FESOM-REcoM Diatom Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        
        DiaChlfesom_surf = DiaChlfesom[:,0]
    
        unitfesom = 'Chl.a [mg m$^{-3}$]'
        labelfesom = 'FESOM-REcoM ({0}-{1})'.format(self.fyear,self.lyear)
        
        # load FESOM Cocco Chl.a data -------------------------------------------------------------------------------------------
        
        from pathlib import Path
        
        cocco_path = Path(self.resultpath + '/CoccoChl.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path = Path(self.resultpath + '/PhaeoChl.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        if cocco_path.is_file():
            
            CoccoChlfesom = pf.get_data(self.resultpath, "CoccoChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
            
            labelfesomCocco = 'FESOM-REcoM Cocco Chl.a {0}-{1}'.format(self.fyear,self.lyear)
            
            CoccoChlfesom_surf = CoccoChlfesom[:,0]

            # load FESOM Phaeo Chl.a data ---------------------------------------------------------------------------------------

            if phaeo_path.is_file():
            
                PhaeoChlfesom = pf.get_data(self.resultpath, "PhaeoChl", years, mesh, 
                                   how="mean", compute=True, runid=self.runname, silent=True)
                
                labelfesomPhaeo = 'FESOM-REcoM Phaeo Chl.a {0}-{1}'.format(self.fyear,self.lyear)
                
                PhaeoChlfesom_surf = PhaeoChlfesom[:,0]
                
                print('4-phytoplankton model is used')

            else:
            
                print('3-phytoplankton model is used')

        
        else:
            
            print('2-phytoplankton model is used')
        
        
        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        PhyChlfesom_surf_interp = pf.fesom2regular(
                data = PhyChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        DiaChlfesom_surf_interp = pf.fesom2regular(
                data = DiaChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        if cocco_path.is_file():
            CoccoChlfesom_surf_interp = pf.fesom2regular(
                data = CoccoChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)

        if phaeo_path.is_file():
            PhaeoChlfesom_surf_interp = pf.fesom2regular(
                data = PhaeoChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        # Nanophyto + Diatoms (+ Coccos / + Phaeos): TOTAL CHLOROPHYLL -------------------------------------------------------------------------------------
        PhyChlfesom_surf_interp_log10 = np.log10(PhyChlfesom_surf_interp)
        DiaChlfesom_surf_interp_log10 = np.log10(DiaChlfesom_surf_interp)
        if cocco_path.is_file():
            CoccoChlfesom_surf_interp_log10 = np.log10(CoccoChlfesom_surf_interp)

        if phaeo_path.is_file():
            PhaeoChlfesom_surf_interp_log10 = np.log10(PhaeoChlfesom_surf_interp)
        
        Chl_total = PhyChlfesom_surf_interp + DiaChlfesom_surf_interp
        if cocco_path.is_file():
            Chl_total = Chl_total + CoccoChlfesom_surf_interp

        if phaeo_path.is_file():
            Chl_total = Chl_total + PhaeoChlfesom_surf_interp
            
        Chl_total_log10 = np.log10(Chl_total)
        
        if False: # interpolation check
            Chl_total_preinterp = ChlAfesom_surf + DiaChlfesom_surf
            
            print('\nFESOM interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
                np.nanmin(Chl_total_preinterp),np.nanmax(Chl_total_preinterp),np.nanmean(Chl_total_preinterp),
                np.nanmin(Chl_total),np.nanmax(Chl_total),np.nanmean(Chl_total)))
        
            fig = plt.figure(figsize=(10,10))
            ax1 = plt.subplot(projection = ccrs.Robinson())

            m1 = plt.pcolormesh(londic, latdic, Chl_total, 
                transform = ccrs.PlateCarree(),
                norm=colors.LogNorm(vmin=np.nanmin(Chl_total), 
                                    vmax=np.nanmax(Chl_total)),
                cmap='viridis')

            ax1.coastlines(resolution='110m', color='black', linewidth=1)

            cbar = fig.colorbar(m1,orientation = 'horizontal',fraction=0.1, pad=0.1) 
            cbar.set_label('Interpolated '+unitfesom, fontsize=20)

        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        OCCCIchla_ma = np.copy(OCCCIchla)
        OCCCIchla_ma[~np.isfinite(Chl_total)] = np.nan
        
        OCCCIchla_ma_log10 = np.log10(OCCCIchla_ma)
        
        # check CHl.a data -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nChl.a\nOCCCI min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f} (Mean over 0 to {4} m)'.format(
                    np.nanmin(Chl_total),np.nanmax(Chl_total),
                    np.nanmin(OCCCIchla_ma),np.nanmax(OCCCIchla_ma),
                    f_depth))

            print('\nlog10(Chl.a)\nOCCCI min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f} (Mean over 0 to {4} m)'.format(
                    np.nanmin(Chl_total_log10),np.nanmax(Chl_total_log10),
                    np.nanmin(OCCCIchla_ma_log10),np.nanmax(OCCCIchla_ma_log10),
                    f_depth))
        
        if(self.plotting):
            # plot each CHl.a dataset -------------------------------------------------------------------------------------        
            
            levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                                   0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                                   1,2,3,4,5])
            ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5]
            ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5'] # +7 ?

            def mygrid(m):
                m.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='gray')


            # if phaeos and coccos are used ----------------------------------------------------------------------------------------
            if cocco_path.is_file() & phaeo_path.is_file():
                
                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        AB
                        CD
                        EF
                        GG
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, PhyChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, DiaChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)

                
                # FESOM coccolithophores
                m3 = axes['C']
                f3 = m3.pcolormesh(londic, latdic, CoccoChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_title('FESOM-REcoM Coccos', fontsize=16)

                # FESOM phaeocystis
                m4 = axes['D']
                f4 = m4.pcolormesh(londic, latdic, PhaeoChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m4)
                m4.set_title('FESOM-REcoM Phaeo', fontsize=16)
                
                
            
                
                # OC-CCI
                m6 = axes['F']
                f1 = m6.pcolormesh(londic, latdic, OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m6)
                m6.set_title('OC-CCI', fontsize=16)



                # FESOM
                m5 = axes['E']
                f5 = m5.pcolormesh(londic, latdic, Chl_total, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m5)
                m5.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label('Chl.a [mg m$^{-3}$]', fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-2,2.125,0.125)
                m7 = axes['G']
                f7 = m7.pcolormesh(londic, latdic, Chl_total - OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256),
                                   #vmin=-3, vmax=3
                                   )
                #f3.set_clim([-2, 2])

                mygrid(m7)
                m7.set_title('FESOM-REcoM - OC-CCI', fontsize=16)

                # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f7,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                #location ='bottom',
                                extend = 'both',
                                ticks = [-3,-2,-1,0,1,2,3],
                                ) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')
                m6.text(-0.12, 1.05, 'F', transform=m6.transAxes,
                            size=30, weight='bold')
                m7.text(-0.12, 1.05, 'G', transform=m7.transAxes,
                            size=30, weight='bold')
                    
                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())
                m6.set_extent(box, ccrs.PlateCarree())
                m7.set_extent(box, ccrs.PlateCarree())
            
            # if coccos are used ----------------------------------------------------------------------------------------
            elif cocco_path.is_file() and not phaeo_path.is_file():
                
                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        ABC
                        DEF
                        GGG
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, PhyChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, DiaChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)

                
                # FESOM coccolithophores
                m3 = axes['C']
                f3 = m3.pcolormesh(londic, latdic, CoccoChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_title('FESOM-REcoM Coccos', fontsize=16)
                
                
                
            
                
                # OC-CCI
                m5 = axes['F']
                f1 = m5.pcolormesh(londic, latdic, OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m5)
                m5.set_title('OC-CCI', fontsize=16)


                
                # Filling gap
                m7 = axes['E']
                axes['E'].remove()
                
                
                
                # FESOM
                m4 = axes['D']
                f4 = m4.pcolormesh(londic, latdic, Chl_total, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m4)
                m4.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label('Chl.a [mg m$^{-3}$]', fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-2,2.125,0.125)
                m6 = axes['G']
                f6 = m6.pcolormesh(londic, latdic, Chl_total - OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256),
                                   #vmin=-3, vmax=3
                                   )
                #f3.set_clim([-2, 2])

                mygrid(m6)
                m6.set_title('FESOM-REcoM - OC-CCI', fontsize=16)

                # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f6,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                #location ='bottom',
                                extend = 'both',
                                ticks = [-3,-2,-1,0,1,2,3],
                                ) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')
                m6.text(-0.12, 1.05, 'F', transform=m6.transAxes,
                            size=30, weight='bold')
                    
                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())
                m6.set_extent(box, ccrs.PlateCarree())

                
            # if coccos and phaeos are not used -------------------------------------------------------------------------------------------    
            else:

                fig = plt.figure(figsize=(15,15), constrained_layout=False)
                axes = fig.subplot_mosaic(
                        """
                        AB
                        CD
                        EE
                        """,
                        gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                        subplot_kw=dict(projection=self.mapproj))             

                # FESOM nanophyto
                m1 = axes['A']
                f1 = m1.pcolormesh(londic, latdic, PhyChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m1)
                m1.set_title('FESOM-REcoM small phytoplankton', fontsize=16)


                # FESOM diatom
                m2 = axes['B']
                f2 = m2.pcolormesh(londic, latdic, DiaChlfesom_surf_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m2)
                m2.set_title('FESOM-REcoM Diatom', fontsize=16)

                # OC-CCI
                m4 = axes['D']
                f1 = m4.pcolormesh(londic, latdic, OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
                mygrid(m4)
                m4.set_title('OC-CCI', fontsize=16)


                # FESOM
                m3 = axes['C']
                f2 = m3.pcolormesh(londic, latdic, Chl_total, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                mygrid(m3)
                m3.set_title('FESOM-REcoM Total', fontsize=16)

                cbar1_ax = fig.add_axes([0.92, 0.44, 0.02, 0.4])

                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'vertical',
                                ticks = ticks,
                                fraction=0.1, pad=0.1,
                                extend = 'max') 
                cbar1.set_label('Chl.a [mg m$^{-3}$]', fontsize=14)
                cbar1.ax.tick_params(labelsize=14)



                # OC-CCI - FESOM
                levels_diff = np.arange(-2,2.125,0.125)
                m5 = axes['E']
                f3 = m5.pcolormesh(londic, latdic, Chl_total - OCCCIchla_ma, 
                                   transform = ccrs.PlateCarree(),
                                   cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256),
                                   #vmin=-3, vmax=3
                                   )
                #f3.set_clim([-2, 2])

                mygrid(m5)
                m5.set_title('FESOM-REcoM - OC-CCI', fontsize=16)

                # add one colorbar for difference plot below figure

                #fig.subplots_adjust(right=0.8)
                cbar2_ax = fig.add_axes([0.92, 0.14, 0.02, 0.2])

                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'vertical',
                                #location ='bottom',
                                extend = 'both',
                                ticks = [-3,-2,-1,0,1,2,3],
                                ) 
                cbar2.ax.tick_params(labelsize=14)
                cbar2.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                m4.text(-0.12, 1.05, 'D', transform=m4.transAxes,
                            size=30, weight='bold')
                m5.text(-0.12, 1.05, 'E', transform=m5.transAxes,
                            size=30, weight='bold')

                m1.set_extent(box, ccrs.PlateCarree())
                m2.set_extent(box, ccrs.PlateCarree())
                m3.set_extent(box, ccrs.PlateCarree())
                m4.set_extent(box, ccrs.PlateCarree())
                m5.set_extent(box, ccrs.PlateCarree())
            
            #plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  

        if(self.Taylor):
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(OCCCIchla_ma_log10)): print('WARNING: OCCCI field contains NaNs')
            if np.isnan(np.min(Chl_total_log10)): print('WARNING: FESOM field contains NaNs')

            # get statistics only from valid OCCCI gridpoints 
            ind_stat = np.where(np.isfinite(OCCCIchla_ma_log10))

            title = 'log10 surface Chlorophyll'
            print('\nStatistics for '+title)
            plt_Taylor_norm(OCCCIchla_ma_log10[ind_stat],Chl_total_log10[ind_stat],
                                    mask=True,title=title)

            
            #plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False) 
        
        if(self.output):
            self.lon = londic
            self.lat = latdic
            self.chl_oc = OCCCIchla_ma
            self.chl_fesom = Chl_total
            self.chld_fesom = DiaChlfesom_surf_interp
            self.chln_fesom = PhyChlfesom_surf_interp
            if cocco_path.is_file():
                self.chlc_fesom = CoccoChlfesom_surf_interp
            if phaeo_path.is_file():
                self.chlp_fesom = PhaeoChlfesom_surf_interp
            
            self.unit = unitfesom